# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [76]:
# Your code here
import pandas as pd 
import sqlite3

conn = sqlite3.connect("data.sqlite") 

## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [78]:
# Your code here
query = """ 
  SELECT firstName, lastName
    FROM employees  
         LEFT JOIN offices 
         USING(officeCode)
         WHERE city = "Boston";
         
"""
pd.read_sql(query, conn)

,firstName,lastName
0,Julie,Firrelli
1,Steve,Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [103]:
# Your code here
query = """ 
  SELECT city, officeCode, 
         COUNT(employees.officeCode)  AS num_of_employees         
  FROM offices 
       LEFT JOIN employees 
       USING(officeCode)        
  GROUP BY officeCode
 HAVING num_of_employees = 0;
"""
pd.read_sql(query, conn)

,city,officeCode,num_of_employees
0,Boston,27,0


## Write 3 questions of your own and answer them

In [107]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
"""
query = """ 
  SELECT o.officeCode,
         o.city,
         COUNT(c.customerNumber) AS num_of_customers
    FROM offices o
       LEFT JOIN employees AS e
       USING(officeCode) 
           LEFT JOIN customers AS c
           ON e.employeeNumber = c.salesRepEmployeeNumber
  GROUP BY officeCode
"""
pd.read_sql(query, conn)



,officeCode,city,num_of_customers
0,1,San Francisco,12
1,2,Boston,12
2,3,NYC,15
3,4,Paris,29
4,5,Tokyo,5
5,6,Sydney,10
6,7,London,17
7,27,Boston,0


In [53]:
"""
Question 1
What are the top 5 countries of customers, from where the highest payment was made? what amount?
"""

# Your code here
pd.read_sql(""" 
    SELECT country, 
           SUM(CAST(amount AS REAL)) AS total_amount_of_payment
      FROM customers
           LEFT JOIN payments
           USING(customerNumber)
    GROUP BY country 
    ORDER BY total_amount_of_payment DESC
    LIMIT 5;
    
""", conn)

,country,total_amount_of_payment
0,USA,3040029.52
1,Spain,994438.53
2,France,965750.58
3,Australia,509385.82
4,New Zealand,392486.59


In [38]:
"""
Question 2
what are the top 5 products with the highest order? show the quantity of their odrer.
"""

# Your code here
pd.read_sql(""" 
    SELECT productName, 
           SUM(quantityOrdered) as quantity_ordered
      FROM products
           LEFT JOIN orderdetails
           USING(productCode)
     GROUP BY productCode
     ORDER BY quantity_ordered DESC
     LIMIT 5;
""", conn)


,productName,quantity_ordered
0,1992 Ferrari 360 Spider red,1808
1,1937 Lincoln Berline,1111
2,American Airlines: MD-11S,1085
3,1941 Chevrolet Special Deluxe Cabriolet,1076
4,1930 Buick Marquette Phaeton,1074


In [40]:
"""
Question 3
who are the top 10 customers with the highest quantity ordered? From which  and what is their total order?
"""

# Your code here
pd.read_sql(""" 
    SELECT customerName, 
           SUM(orderdetails.quantityOrdered) AS quantity_ordered
      FROM customers
           LEFT JOIN orders
               USING(customerNumber)
           LEFT JOIN orderdetails
               USING(orderNumber)
     GROUP BY customerNumber
     ORDER BY quantity_ordered DESC
     LIMIT 10            
""", conn)

,customerName,quantity_ordered
0,Euro+ Shopping Channel,9327
1,Mini Gifts Distributors Ltd.,6366
2,"Australian Collectors, Co.",1926
3,La Rochelle Gifts,1832
4,"AV Stores, Co.",1778
5,Muscle Machine Inc,1775
6,"Down Under Souveniers, Inc",1691
7,The Sharp Gifts Warehouse,1656
8,Rovelli Gifts,1650
9,Kelly's Gift Shop,1647


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [56]:
# Your code here
pd.read_sql(""" 
    SELECT firstName, lastName, productName
      FROM employees
           LEFT JOIN customers 
           ON employees.employeeNumber = customers.salesRepEmployeeNumber
               LEFT JOIN orders 
               USING(customerNumber)
                   LEFT JOIN orderdetails
                   USING(orderNumber) 
                       LEFT JOIN products
                       USING(productCode)
    WHERE productName IS NOT NULL
    GROUP BY employees.employeeNumber;
    
    
""", conn)


,firstName,lastName,productName
0,Leslie,Jennings,1958 Setra Bus
1,Leslie,Thompson,1917 Grand Touring Sedan
2,Julie,Firrelli,1952 Alpine Renault 1300
3,Steve,Patterson,2001 Ferrari Enzo
4,Foon Yue,Tseng,2001 Ferrari Enzo
5,George,Vanauf,1969 Harley Davidson Ultimate Chopper
6,Loui,Bondur,1952 Alpine Renault 1300
7,Gerard,Hernandez,1965 Aston Martin DB5
8,Pamela,Castillo,1972 Alfa Romeo GTA
9,Larry,Bott,1972 Alfa Romeo GTA


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [62]:
# Your code here
pd.read_sql(""" 
    SELECT firstName, lastName, productName, 
           SUM(orderdetails.quantityOrdered) AS num_of_products_sold
      FROM employees
           LEFT JOIN customers 
           ON employees.employeeNumber = customers.salesRepEmployeeNumber
               LEFT JOIN orders 
               USING(customerNumber)
                   LEFT JOIN orderdetails
                   USING(orderNumber) 
                       LEFT JOIN products
                       USING(productCode)
   WHERE productName IS NOT NULL
    GROUP BY employees.employeeNumber
    ORDER BY lastName;
    
    
""", conn)


,firstName,lastName,productName,num_of_products_sold
0,Loui,Bondur,1952 Alpine Renault 1300,6186
1,Larry,Bott,1972 Alfa Romeo GTA,8205
2,Pamela,Castillo,1972 Alfa Romeo GTA,9290
3,Julie,Firrelli,1952 Alpine Renault 1300,4227
4,Andy,Fixter,1996 Moto Guzzi 1100i,6246
5,Martin,Gerard,The Titanic,4180
6,Gerard,Hernandez,1965 Aston Martin DB5,14231
7,Leslie,Jennings,1958 Setra Bus,11854
8,Barry,Jones,1952 Alpine Renault 1300,7486
9,Peter,Marsh,1962 LanciaA Delta 16V,6632


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [65]:
# Your code here
# Your code here
pd.read_sql(""" 
    SELECT firstName, lastName,
           COUNT(products.productCode) AS different_products_sold
      FROM employees
           LEFT JOIN customers 
           ON employees.employeeNumber = customers.salesRepEmployeeNumber
               LEFT JOIN orders 
               USING(customerNumber)
                   LEFT JOIN orderdetails
                   USING(orderNumber) 
                       LEFT JOIN products
                       USING(productCode)
    GROUP BY employees.employeeNumber
    HAVING different_products_sold > 200;
    
    
""", conn)


,firstName,lastName,different_products_sold
0,Leslie,Jennings,331
1,George,Vanauf,211
2,Gerard,Hernandez,396
3,Pamela,Castillo,272
4,Larry,Bott,236
5,Barry,Jones,220


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!